In [1]:
SANDBOX_NAME = '' # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Spark ML Clustering

Haremos un **Análisis RFM** antes de aplicar un algoritmo de clustering.

**RFM** es un método usado para analizar el valor del cliente. Es habitualmente utilizado en bases de datos de marketing y marketing directo y recibe atención especial en los sectores de retail y servicios.

Las siglas RFM significan las tres dimensiones:

* Recency - *¿Cuánto tiempo hace desde la última compra del cliente?*

* Frequency - *¿Con qué frecuencia compra el cliente?*

* Monetary Value - *¿Cuánto gasta el cliente?*

RFM (customer value). (s.f.). En *Wikipedia*. Recuperado el 29 de Enero de 2019 de [https://en.wikipedia.org/wiki/RFM_(customer_value)](https://en.wikipedia.org/wiki/RFM_(customer_value))



### Crear SparkSession

In [2]:
# Respuesta aqui



### Cargar datos y comprobar schema

En este caso se utilizará el dataset 'Online Retail.csv'

In [3]:
# Respuesta aqui

In [4]:
# Respuesta aqui



#### Pasos previos

* Modificamos los tipos de las columnas CustomerID y de UnitPrice. UnitPrice debe sustituirse la coma por punto en los numeros decimales para que tenga formato float. 

In [5]:
# Respuesta aqui



Luego agrupamos por país y vemos las operaciones totales realizadas por país. Esto lo vamos a realizar así para sesgar por el país que más operaciones tenga.

In [6]:
# Respuesta aqui

In [7]:
# Respuesta aqui



* Vemos que la mayoría del histórico de los datos están en Reino Unido, restringimos los datos solo a los clientes de Reino Unido.

In [8]:
# Respuesta aqui



* Eliminar todos los nulos

In [9]:
# Respuesta aqui



* Eliminar pedidos cancelados. Es decir, debemos quedarnos con aquellos registros donde la columna "Quantity" sea mayor que cero.

In [10]:
# Respuesta aqui



* Añadimos la columna precio total, debe llamarse "TotalPrice". Debe ser "Quantity" x "UnitPrice" 

In [11]:
# Respuesta aqui



* Creamos una nueva columna, de formato _timestamp_, en base a la columna InvoiceDate 

In [12]:
# Respuesta aqui



* Cálculo de Recency. Para el cálculo de recency asumimos que la fecha actual es el 30/12/2011

In [13]:
# Respuesta aqui

In [14]:
sales_uk.orderBy('DaysBefore', ascending=True).show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+----------+-------------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|TotalPrice|     NewInvoiceDate|DaysBefore|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+----------+-------------------+----------+
|   581475|    23239|SET OF 4 KNICK KN...|       6|09/12/2011 8:39|     1.65|     13069|United Kingdom|       9.9|2011-12-09 08:39:00|        21|
|   581475|    23235|STORAGE TIN VINTA...|      12|09/12/2011 8:39|     1.25|     13069|United Kingdom|      15.0|2011-12-09 08:39:00|        21|
|   581475|    23272|TREE T-LIGHT HOLD...|      12|09/12/2011 8:39|     0.39|     13069|United Kingdom|      4.68|2011-12-09 08:39:00|        21|
|   581475|    22596|CHRISTMAS STAR WI...|      36|09/12/2011 8:39|     0.39|     13069|United Kingdom|     14.04|2011-12-09

In [15]:
# Respuesta aqui

In [16]:
# Respuesta aqui



* Cálculo de Frequency.

In [17]:
# Respuesta aqui

In [18]:
# Respuesta aqui



* Cálculo de Monetary.

In [19]:
# Respuesta aqui

In [20]:
# Respuesta aqui

In [21]:
# Respuesta aqui



* Unimos los valores de RFM por CustomerID

In [22]:
# Respuesta aqui

In [23]:
rfm_df.show(5)

+----------+-------+---------+--------+
|CustomerID|Recency|Frequency|Monetary|
+----------+-------+---------+--------+
|     12847|     43|        1|  871.54|
|     13192|    116|        2|  911.94|
|     13282|     39|        3| 1132.14|
|     13610|     33|        7| 1131.88|
|     13772|     54|        3| 1132.13|
+----------+-------+---------+--------+
only showing top 5 rows





Vamos a realizar el estudio de clustering con este nuevo dataset generado `rfm_df`

* VectorAssembler con todas las variables disponibles: `Recency`, `Frequency`, `Monetary`

In [24]:
# Respuesta aqui



* StandardScaler con variables cuantitativas ('Recency', 'Frequency', 'Monetary')

In [25]:
# Respuesta aqui

In [26]:
rfm_df.show(5)

+----------+-------+---------+--------+------------------+--------------------+
|CustomerID|Recency|Frequency|Monetary|assembled_features|     features_scaled|
+----------+-------+---------+--------+------------------+--------------------+
|     12847|     43|        1|  871.54| [43.0,1.0,871.54]|[0.43203691518006...|
|     13192|    116|        2|  911.94|[116.0,2.0,911.94]|[1.16549493397413...|
|     13282|     39|        3| 1132.14|[39.0,3.0,1132.14]|[0.39184743469820...|
|     13610|     33|        7| 1131.88|[33.0,7.0,1131.88]|[0.33156321397540...|
|     13772|     54|        3| 1132.13|[54.0,3.0,1132.13]|[0.54255798650520...|
+----------+-------+---------+--------+------------------+--------------------+
only showing top 5 rows





### KMeans



* Se debe estudiar cual es el mejor valor de K (utilizando el dataframe `rfm_df`), es decir, cuál es el número de clusters óptimo para este conjunto de datos

In [27]:
# Respuesta aqui

In [28]:
# Respuesta aqui



* La técnica del codo

Es común realizar un gráfico 2D para saber qué valor de K elegir. Normalmente en las absisas se coloca el número de clusters y en la ordenadas se suele colocar un métrica  de costo que permita cuantificar la calidad de los clusters, por ejemplo: la suma de las distancias al cuadrado de los puntos al centro más cercano (ver la documentacion con el método _computeCost_).
A mayor número de K, la métrica de costo tiene a disminuir. Se debe elegir aquel máximo valor de K tenga una disminución significativa del costo.

En este caso particular se debería apreciar que pasar de K=7 a un K=8 disminuye a la mitad el costo. Sin embargo, aumentar el valor de K=9 no tiene un impacto significativo en el costo. Por estas razones deberíamos elegir K = 8.

In [29]:
# Respuesta aqui



* Vemos cuántos elementos hay en cada cluster.

In [30]:
# Respuesta aqui



¿Qué piensa de los grupos y su distribución?



* Calculamos la media de los valores de RFM para cada cluster para ver las características diferenciales de cada uno de los clusters.

In [31]:
# Respuesta aqui



### BisectingKMeans

In [32]:
# Respuesta aqui



* Vemos la frecuencia de cada cluster.

In [33]:
# Respuesta aqui



* Calculamos la media de los valores de RFM para cada cluster.

In [34]:
# Respuesta aqui



#### Opcional

Existen varias métricas para comparar entre clusterings, a pesar de que en función de la aplicación, la información que arrojen las variables incluidas en el estudio, y la calidad de los datos, los resultados varían considerablemente. 

La métrica de la silueta oscila entre -1 y 1, y cuanto más cercano esté a 1, mejor se puede llegar a considerar el clustering. Dado que mide las distancias que existen entre clustering, cuando más diferenciados estén entre sí, mejor se puede decir que se ha segmentado la población.

In [37]:
# Respuesta aqui

In [38]:
# Respuesta aqui

In [39]:
# Respuesta aqui

In [40]:
# Respuesta aqui